In [2]:
#save frozen model

# import the needed libraries
import tensorflow as tf
import tensorflow.contrib.tensorrt as trt
from tensorflow.python.platform import gfile
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

with tf.Session(config=tf.ConfigProto(gpu_options=tf.GPUOptions(per_process_gpu_memory_fraction=0.50))) as sess:
    sess.run(tf.global_variables_initializer())
    saver = tf.train.import_meta_graph(os.path.join('model', 'model.meta'))
    saver.restore(sess, tf.train.latest_checkpoint('model'))
    your_outputs = ['X','Y','generator/xs']
    frozen_graph = tf.graph_util.convert_variables_to_constants(
        sess, # session
        tf.get_default_graph().as_graph_def(),# graph+weight from the session
        output_node_names=your_outputs)
    #write the TensorRT model to be used later for inference
    with gfile.FastGFile("./model/frozen_model.pb", 'wb') as f:
        f.write(frozen_graph.SerializeToString())
    print("Frozen model is successfully stored!")

W0427 09:54:32.875017 140140661520192 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W0427 09:54:35.858277 140140661520192 deprecation.py:323] From <ipython-input-2-9dade628b135>:16: convert_variables_to_constants (from tensorflow.python.framework.graph_util_impl) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`
W0427 09:54:35.858875 140140661520192 deprecation.py:323] From /usr/local/lib/python3.6/site-packages/tensorflow_core/python/framework/graph_util_impl.py:277: extract_sub_graph (from tensorflow.python.framework.graph_util_impl) is deprecated and will be r

Frozen model is successfully stored!


In [2]:

# import the needed libraries
import tensorflow as tf
import tensorflow.contrib.tensorrt as trt
from tensorflow.python.platform import gfile
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

W0427 15:33:05.735058 139666054952768 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [4]:
def get_frozen_graph(graph_file):
  """Read Frozen Graph file from disk."""
  with tf.gfile.FastGFile(graph_file, "rb") as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())
  return graph_def
frozen_graph_def = get_frozen_graph('model/frozen_model.pb')
print(type(frozen_graph_def))

<class 'tensorflow.core.framework.graph_pb2.GraphDef'>


In [ ]:
output_nodes = ['X','Y','generator/xs']
output_dir = 'tensorrt_dir'

trt_graph_def = trt.create_inference_graph(
    frozen_graph_def,
    output_nodes,
    max_batch_size=1,
    max_workspace_size_bytes=(2 << 10) << 20,
    precision_mode='INT8')

tf.reset_default_graph()
g = tf.Graph()

with g.as_default():
    tf.import_graph_def(
      graph_def=trt_graph_def,
      return_elements=output_nodes,
      name=''
    )

with tf.Session(graph=g) as sess:
    builder = tf.saved_model.builder.SavedModelBuilder(output_dir)
    builder.add_meta_graph_and_variables(
      sess,
      [tf.saved_model.tag_constants.SERVING]
    )
    builder.save()
    train_writer = tf.summary.FileWriter(output_dir)
    train_writer.add_graph(sess.graph)
#/opt/ipynbs/work/beauty/tensorrt_dir/saved_model.pb 